<a href="https://colab.research.google.com/github/biamouras/alura_pyspark_challenge/blob/main/Notebooks/02_tratamento_dados_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Insightplaces - Tratamento dos dados para ML

A Insightplaces, plataforma fantasia de imóveis da Alura, está com dificuldades para definir os valores do imóveis e gostaria de fazer de modo automatizado. Ainda, a empresa percebeu que o recomendador atual não está sendo efetivo para gerar novos cliques entre os usuários.

Nosso projeto será trabalhar com os dados disponibilizados de imóveis no Rio de Janeiro para ajudar a empresa a lidar com estes dois problemas usando regressão para a definição dos valores e a análise de agrupamentos através de Machine Learning. 

Este segundo projeto tem o foco de preparar os dados para os modelos de previsão a partir de dados já tratados.

## Carregando os dados

In [1]:
from os import environ, listdir

In [4]:
# instalar as dependências para utilizar Spark no Colab
instala = True

# verifica se já está instalado 
for f in listdir('/content'):
  if f.find('spark') != -1:
    instala = False
    break 

if instala:
  !apt-get update -qq
  !apt-get install openjdk-8-jdk-headless -qq > /dev/null

  !wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
  !tar xf spark-3.2.1-bin-hadoop3.2.tgz
  !pip install -q findspark

environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

import findspark
findspark.init()

In [5]:
# iniciando o pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050')\
    .getOrCreate()

In [6]:
# monta a conexão com o drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# biblioteca para abrir o arquivo compactado
import zipfile

In [39]:
# caminho do arquivo 
name = 'semana-2'
dir_dados = '/content/drive/MyDrive/Colab Notebooks/alura_challenge_insightplaces/dados/' + name + '/'

# descompressão do arquivo
zipfile.ZipFile(dir_dados + name + '.zip', 'r').extractall(dir_dados)
# identificação do arquivo descomprimido
files = listdir(dir_dados)
files 

['semana-2.zip', 'dataset_transformado_parquet']

In [40]:
# o zip contém uma pasta
dir_dados_originais = dir_dados + files[1]
# verifica os arquivos dentro da pasta comprimida
files = listdir(dir_dados_originais)
files

['._SUCCESS.crc',
 '_SUCCESS',
 'part-00000-00341ba7-0a7c-4fef-a81e-1066725a64b1-c000.snappy.parquet',
 '.part-00000-00341ba7-0a7c-4fef-a81e-1066725a64b1-c000.snappy.parquet.crc']

In [41]:
# leitura do parquet
parquet_file = [f for f in files if f.endswith('.parquet')][0]
dados = spark.read.parquet(dir_dados_originais + '/' + parquet_file)
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



## Correção do tipo de  variável

Analisando os tipos de dados das colunas, vemos que features que deveriam ser numéricas estão consideradas como `string`. 

In [42]:
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



In [12]:
import pyspark.sql.types as types

In [43]:
# convertendo area_total, area_util, condominio, iptu e valor
double_cols = ['area_total', 'area_util', 'condominio', 'iptu', 'valor']

for col in double_cols:
  dados = dados\
          .withColumn(col, dados[col].cast(types.DoubleType()))

dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: double (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: double (nullable = true)



Outras variáveis, como `andar`, `banheiros`, `quartos`, `suites` e `vagas`, estão como `long` e podem ocupar muito espaço na memória sem necessidade. Portanto, vamos convertê-las para ShortType.

In [44]:
# colunas para ShortType
short_cols = ['andar', 'banheiros', 'quartos', 'suites', 'vaga']

# verificando limites das variáveis
dados\
  .select(short_cols)\
  .summary()\
  .show()

+-------+------------------+------------------+------------------+------------------+------------------+
|summary|             andar|         banheiros|           quartos|            suites|              vaga|
+-------+------------------+------------------+------------------+------------------+------------------+
|  count|             66562|             66562|             66562|             61008|             63545|
|   mean|2.3374598119046905| 2.454583696403353|2.6121811243652533|1.2091528979805928|1.4069084900464237|
| stddev|  14.6963496227043|1.3582533594407473|0.9083944981999731|1.0567921717756774|1.3413027342394985|
|    min|                 0|                 1|                 0|                 0|                 0|
|    25%|                 0|                 2|                 2|                 1|                 1|
|    50%|                 0|                 2|                 3|                 1|                 1|
|    75%|                 3|                 3|        

Como os valores máximos não ultrapassam o limite de ShortType (entre -32868 a 32767), podemos realizar a conversão.

In [45]:
for col in short_cols:
  dados = dados\
                .withColumn(col, dados[col].cast(types.ShortType()))

In [46]:
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: short (nullable = true)
 |-- area_total: double (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: short (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: short (nullable = true)
 |-- suites: short (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: short (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: double (nullable = true)



## Tratamento de valores Nulos

In [17]:
import pyspark.sql.functions as f

Analisando a quantidade de observações válidas (66.562), vemos que as features `area_total`, `suites`, `vaga`, `condominio`, e `iptu` têm menos registros, indicando a presença de observações nulas.

In [47]:
# seleciona variáveis numéricas
var_numericas = [var for var, types in dados.dtypes if 'string' not in types]

# identifica a quantidade de NA em cada coluna
dados\
  .select(var_numericas)\
  .summary()\
  .show()

+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|summary|             andar|       area_total|         area_util|         banheiros|           quartos|            suites|              vaga|        condominio|              iptu|             valor|
+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|  count|             66562|            57368|             66562|             66562|             66562|             61008|             63545|             64191|             59363|             66562|
|   mean|2.3374598119046905|938.1081264816622|116.73728253357771| 2.454583696403353|2.6121811243652533|1.2091528979805928|1.4069084900464237|4822.7975728684705|5447.7366036083085|1294433.2853129413|
| std

De modo genérico, vamos preencher os valores nulos pelas médias de cada variável em relação aos bairros.

In [48]:
from pyspark.sql import Window

# variáveis que precisam de preenchimento
var_na = ['area_total', 'suites', 'vaga', 'condominio', 'iptu']

# calcula as médias para cada feature (criar um dicionário)
w = Window.partitionBy(dados.bairro)

df = dados

for col in var_na:
  df = df\
  .withColumn(col, 
              f.when(f.isnan(col) | f.isnull(col), 
                          f.avg(col).over(w))\
              .otherwise(dados[col]))

df.select(var_na + ['bairro']).show(10)

+-----------------+-------------------+------------------+----------+-----+--------+
|       area_total|             suites|              vaga|condominio| iptu|  bairro|
+-----------------+-------------------+------------------+----------+-----+--------+
|             88.0|0.11764705882352941|               1.0|     700.0|950.0|Abolição|
|             44.0|                0.0|               1.0|     755.0|  0.0|Abolição|
|             52.0|                0.0|               1.0|     420.0|158.7|Abolição|
|             34.0|0.11764705882352941|0.8636363636363636|     243.0|158.7|Abolição|
|             90.0|0.11764705882352941|0.8636363636363636|     760.0|  0.0|Abolição|
|             43.0|                0.0|               1.0|     500.0| 16.0|Abolição|
|             53.0|0.11764705882352941|               1.0|     473.0|  0.0|Abolição|
|             60.0|                0.0|               1.0|     250.0|158.7|Abolição|
|59.65384615384615|                0.0|               1.0|     56

In [49]:
# verifica a quantidade de NA em cada coluna tratada
df\
  .select(var_na)\
  .summary()\
  .show()

+-------+-----------------+------------------+------------------+------------------+------------------+
|summary|       area_total|            suites|              vaga|        condominio|              iptu|
+-------+-----------------+------------------+------------------+------------------+------------------+
|  count|            66562|             66558|             66559|             66560|             66556|
|   mean|966.3603330598665|1.1766074024133613|1.3883332557343786| 4795.622909567849| 5393.876817238429|
| stddev|78149.95419995103|1.0254394312575297|1.3158729395552726|104147.68733138188|171136.61673109749|
|    min|              0.0|               0.0|               0.0|               0.0|               0.0|
|    25%|             70.0|               1.0|               1.0|             476.0|              80.0|
|    50%|             97.0|               1.0|               1.0|             900.0|             482.0|
|    75%|            154.0|               1.0|               2.0

## Seleção de features

Devido a alguns filtros realizados nos dados brutos em relação as colunas: `tipo_uso`, `tipo_unidade` e `tipo_anuncio`, podemos desconsiderá-las para os modelos.

Antes, vamos verificar se de fatos os valores para estas colunas são únicos.

In [50]:
# verificação da variabilidade
df.createOrReplaceTempView('dfView')
spark\
  .sql("""
        SELECT DISTINCT tipo_uso, tipo_unidade, tipo_anuncio, tipo
        FROM dfView
  """)\
  .show()

+-----------+------------+------------+-----+
|   tipo_uso|tipo_unidade|tipo_anuncio| tipo|
+-----------+------------+------------+-----+
|Residencial| Apartamento|       Usado|Venda|
+-----------+------------+------------+-----+



In [51]:
df = df.drop('tipo_uso', 'tipo_unidade', 'tipo_anuncio', 'tipo')
df.show(5)

+--------------------+-----+----------+---------+---------+--------------------+-------+-------------------+------------------+--------+----------+----------+-----+--------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|             suites|              vaga|  bairro|      zona|condominio| iptu|   valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+-------------------+------------------+--------+----------+----------+-----+--------+
|5c0573ed-a0b6-454...|    5|      88.0|     88.0|        1|[Churrasqueira, E...|      2|0.11764705882352941|               1.0|Abolição|Zona Norte|     700.0|950.0|309600.0|
|ec4bb157-f2da-4fc...|    0|      44.0|     44.0|        1|[Churrasqueira, E...|      2|                0.0|               1.0|Abolição|Zona Norte|     755.0|  0.0|175000.0|
|4cd3efdd-f835-4cc...|    0|      52.0|     52.0|        1|[Churrasqueira, P...|      2|                0.0|               1.0|Abo

Temos ainda as variáveis `area_total` e `area_util`. Considerando que houve a necessidade de tratamento da `area_total` pela falta de informação, e que há grande variabilidade entre os valores desta variável, também vamos descartá-la.

In [52]:
df\
  .select(['area_total', 'area_util'])\
  .summary()\
  .show()

+-------+-----------------+------------------+
|summary|       area_total|         area_util|
+-------+-----------------+------------------+
|  count|            66562|             66562|
|   mean|966.3603330598665|116.73728253357771|
| stddev|78149.95419995103| 89.09094364150437|
|    min|              0.0|              10.0|
|    25%|             70.0|              67.0|
|    50%|             97.0|              90.0|
|    75%|            154.0|             138.0|
|    max|      1.5022793E7|            3456.0|
+-------+-----------------+------------------+



In [53]:
df = df.drop('area_total')
df.show(5)

+--------------------+-----+---------+---------+--------------------+-------+-------------------+------------------+--------+----------+----------+-----+--------+
|                  id|andar|area_util|banheiros|     caracteristicas|quartos|             suites|              vaga|  bairro|      zona|condominio| iptu|   valor|
+--------------------+-----+---------+---------+--------------------+-------+-------------------+------------------+--------+----------+----------+-----+--------+
|5c0573ed-a0b6-454...|    5|     88.0|        1|[Churrasqueira, E...|      2|0.11764705882352941|               1.0|Abolição|Zona Norte|     700.0|950.0|309600.0|
|ec4bb157-f2da-4fc...|    0|     44.0|        1|[Churrasqueira, E...|      2|                0.0|               1.0|Abolição|Zona Norte|     755.0|  0.0|175000.0|
|4cd3efdd-f835-4cc...|    0|     52.0|        1|[Churrasqueira, P...|      2|                0.0|               1.0|Abolição|Zona Norte|     420.0|158.7|205000.0|
|1c72e89a-bba5-4fc...|

## Transformação das categóricas em `dummies`

Temos três featueres que podem ser convertidas para dummies: `caracteristicas`, `bairro` e `zona`. Até o momento, a função `ps.get_dummies` não converte listas, então terei que trabalhar com a `caracteristicas` em separado.

In [26]:
import pyspark.pandas as ps

In [54]:
# seleção das variáveis categóricas
var_cat = ['bairro', 'zona']

df_cat = df.select(var_cat + ['id'])
df_cat = ps.DataFrame(df_cat) # é necessário converter para o DF de ps

df_dummies = ps.get_dummies(
    df_cat,
    columns = var_cat,
    prefix = ['bair', 'zona'],
    dummy_na = False,
    drop_first = True
)

df_dummies.head()

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series


,id,bair_Alto da Boa Vista,bair_Anchieta,bair_Andaraí,bair_Anil,bair_Bancários,bair_Bangu,bair_Barra da Tijuca,bair_Barra de Guaratiba,bair_Barros Filho,bair_Benfica,bair_Bento Ribeiro,bair_Boa Esperança,bair_Bonsucesso,bair_Botafogo,bair_Braz de Pina,bair_Cachambi,bair_Cacuia,bair_Caju,bair_Camorim,bair_Campinho,bair_Campo Grande,bair_Campo dos Afonsos,bair_Cascadura,bair_Catete,bair_Catumbi,bair_Cavalcanti,bair_Centro,bair_Cidade Nova,bair_Cidade de Deus,bair_Cocotá,bair_Coelho Neto,bair_Colégio,bair_Copacabana,bair_Cordovil,bair_Cosme Velho,bair_Cosmos,bair_Curicica,bair_Del Castilho,bair_Deodoro,bair_Encantado,bair_Engenheiro Leal,bair_Engenho Novo,bair_Engenho da Rainha,bair_Engenho de Dentro,bair_Engenho do Porto,bair_Estácio,bair_Flamengo,bair_Freguesia (Ilha do Governador),bair_Freguesia (Jacarepaguá),bair_Galeão,bair_Gamboa,bair_Gardênia Azul,bair_Glória,bair_Grajaú,bair_Grande Rio,bair_Guadalupe,bair_Guaratiba,bair_Gávea,bair_Higienópolis,bair_Honório Gurgel,bair_Humaitá,bair_Inhaúma,bair_Inhoaíba,bair_Ipanema,bair_Irajá,bair_Itanhangá,bair_Jacarepaguá,bair_Jacaré,bair_Jardim América,bair_Jardim Botânico,bair_Jardim Carioca,bair_Jardim Guanabara,bair_Jardim Sulacap,bair_Joá,bair_Lagoa,bair_Laranjeiras,bair_Leblon,bair_Leme,bair_Lins de Vasconcelos,bair_Madureira,bair_Magalhães Bastos,bair_Mangueira,bair_Manguinhos,bair_Maracanã,bair_Marechal Hermes,bair_Maria da Graça,bair_Moneró,bair_Méier,bair_Olaria,bair_Oswaldo Cruz,bair_Paciência,bair_Padre Miguel,bair_Paquetá,bair_Parada de Lucas,bair_Parque Anchieta,bair_Parque Colúmbia,bair_Parque da Matriz,bair_Pavuna,bair_Pechincha,bair_Pedra de Guaratiba,bair_Penha,bair_Penha Circular,bair_Piedade,bair_Pilares,bair_Pitangueiras,bair_Portuguesa,bair_Praia da Bandeira,bair_Praça Seca,bair_Praça da Bandeira,bair_Quintino Bocaiúva,bair_Ramos,bair_Realengo,bair_Recreio dos Bandeirantes,bair_Riachuelo,bair_Ribeira,bair_Ricardo de Albuquerque,bair_Rio Comprido,bair_Rocha,bair_Rocha Miranda,bair_Sampaio,bair_Santa Cruz,bair_Santa Teresa,bair_Santo Cristo,bair_Santíssimo,bair_Saúde,bair_Senador Camará,bair_Senador Vasconcelos,bair_Sepetiba,bair_São Conrado,bair_São Cristóvão,bair_São Francisco Xavier,bair_Tanque,bair_Taquara,bair_Tauá,bair_Tijuca,bair_Todos os Santos,bair_Tomás Coelho,bair_Turiaçu,bair_Urca,bair_Vargem Grande,bair_Vargem Pequena,bair_Vasco da Gama,bair_Vaz Lobo,bair_Vicente de Carvalho,bair_Vidigal,bair_Vigário Geral,bair_Vila Isabel,bair_Vila Kosmos,bair_Vila Valqueire,bair_Vila da Penha,bair_Vista Alegre,bair_Zumbi,bair_Água Santa,zona_Zona Central,zona_Zona Norte,zona_Zona Oeste,zona_Zona Sul
0,03a386b6-7ab8-4eff-891d-f8a16efc1989,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1fe78d41-b8e0-4d21-a1a8-72cb7863f441,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,1fa1c1e5-e98c-433f-9675-1b43d7d6da01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,a6ab01ae-3d40-40e7-bb6a-f2334e0abbe0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,

### Coluna `caracteristicas`

A coluna `caracteristicas` tem diferentes especificidades que exigem um tratamento à parte. Esta coluna contém um array com características do condomínio do imóvel, como elevador, churrasqueira e piscina.

In [60]:
# pega coluna de características
df_carac = df.select('id', 'caracteristicas')

# concatena a lista dentro de características 
df_carac = df_carac\
      .withColumn('aux_carac', f.concat_ws(', ', 'caracteristicas'))\
      .filter()

df_carac.show(10)   


+--------------------+--------------------+--------------------+
|                  id|     caracteristicas|           aux_carac|
+--------------------+--------------------+--------------------+
|03a386b6-7ab8-4ef...|[Churrasqueira, A...|Churrasqueira, Ac...|
|1fe78d41-b8e0-4d2...|                  []|                    |
|1fa1c1e5-e98c-433...|                  []|                    |
|a6ab01ae-3d40-40e...|                  []|                    |
|1d8ff7a2-16bd-4e3...|                  []|                    |
|40a2cbde-946b-42d...|[Condomínio fecha...|Condomínio fechad...|
|be269ddf-a4a3-4a7...|[Churrasqueira, C...|Churrasqueira, Co...|
|4b7e3d59-34fd-4ba...|[Churrasqueira, P...|Churrasqueira, Pi...|
|a1a0c2ac-4fa8-4b0...|[Churrasqueira, E...|Churrasqueira, El...|
|390d2b88-e06d-47e...|   [Salão de festas]|     Salão de festas|
+--------------------+--------------------+--------------------+
only showing top 10 rows



In [59]:


# junta todo mundo em um string ou lista
caracteristicas = df_carac.agg(f.concat_ws(", ", f.collect_list(df_carac.caracteristicas)))
caracteristicas

AnalysisException: ignored

In [34]:
# pegar os uniques
# cria manualmente as variáveis usando find